In [1]:
import pandas as pd
import numpy as np
import preprocessing as pre
from ast import literal_eval
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import re

In [2]:
df = pd.read_csv('dataset/movies_metadata.csv')
df.head()

C:\Users\Sveta\AppData\Local\Temp\ipykernel_8736\1729101631.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('dataset/movies_metadata.csv')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [3]:
cred_df = pd.read_csv('dataset/credits.csv')
key_df = pd.read_csv('dataset/keywords.csv')
key_df.shape, cred_df.shape

((46419, 2), (45476, 3))

In [4]:
import preprocessing as pre
df = pre.clean_data(df)
df

c:\Users\Sveta\projects\preprocessing.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
c:\Users\Sveta\projects\preprocessing.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year'] = pd.DatetimeIndex(df['release_date']).year


,title,genres,runtime,vote_average,vote_count,overview,id,year,score
0,Toy Story,"[animation, comedy, family]",81.0,7.7,5415.0,"Led by Woody, Andy's toys live happily in his ...",862,1995.0,7.680953
1,Jumanji,"[adventure, fantasy, family]",104.0,6.9,2413.0,When siblings Judy and Peter discover an encha...,8844,1995.0,6.873979
2,Grumpier Old Men,"[romance, comedy]",101.0,6.5,92.0,A family wedding reignites the ancient feud be...,15602,1995.0,6.189510
4,Father of the Bride Part II,[comedy],106.0,5.7,173.0,Just when George Banks has recovered from his ...,11862,1995.0,5.681661
5,Heat,"[action, crime, drama, thriller]",170.0,7.7,1886.0,"Obsessive master thief, Neil McCauley leads a ...",949,1995.0,7.646235
...,...,...,...,...,...,...,...,...,...
45339,With Open Arms,[comedy],92.0,5.2,94.0,Jean-Étienne Fougerole is an intellectual bohe...,430365,2017.0,5.345211
45343,The Visitors: Bastille Day,[comedy],110.0,4.0,167.0,"Stuck in the corridors of time, Godefroy de Mo...",248705,2016.0,4.372859
45348,Titanic 2,"[action, adventure, thriller]",90.0,3.4,55.0,On the 100th anniversary of the original voyag...,44918,2010.0,4.456289
45437,In a Heartbeat,"[family, animation, romance, comedy]",4.0,8.3,146.0,A closeted boy runs the risk of being outed by...,455661,2017.0,7.615869


In [5]:
def clean_ids(x):
    try:
        return int(x)
    except:
        return np.nan
    
df['id'] = df['id'].apply(clean_ids)
df = df[df['id'].notnull()] 

In [6]:
df['id'] = df['id'].astype('int')
key_df['id'] = key_df['id'].astype('int')
cred_df['id'] = cred_df['id'].astype('int')

# давайте объединим ключевые слова и титры в наш обновленный фрейм метаданных
df = df.merge(cred_df, on='id')
df = df.merge(key_df, on='id')
df.head()

,title,genres,runtime,vote_average,vote_count,overview,id,year,score,cast,crew,keywords
0,Toy Story,"[animation, comedy, family]",81.0,7.7,5415.0,"Led by Woody, Andy's toys live happily in his ...",862,1995.0,7.680953,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,Jumanji,"[adventure, fantasy, family]",104.0,6.9,2413.0,When siblings Judy and Peter discover an encha...,8844,1995.0,6.873979,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,Grumpier Old Men,"[romance, comedy]",101.0,6.5,92.0,A family wedding reignites the ancient feud be...,15602,1995.0,6.189510,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,Father of the Bride Part II,[comedy],106.0,5.7,173.0,Just when George Banks has recovered from his ...,11862,1995.0,5.681661,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
4,Heat,"[action, crime, drama, thriller]",170.0,7.7,1886.0,"Obsessive master thief, Neil McCauley leads a ...",949,1995.0,7.646235,"[{'cast_id': 25, 'character': 'Lt. Vincent Han...","[{'credit_id': '52fe4292c3a36847f802916d', 'de...","[{'id': 642, 'name': 'robbery'}, {'id': 703, '..."


In [7]:
obj_col = list(df.select_dtypes(include=['object']))
obj_col

['title', 'genres', 'overview', 'cast', 'crew', 'keywords']

In [8]:
features = ['cast', 'crew', 'keywords']
for feature in features:
    df[feature] = df[feature].apply(literal_eval)
df.head()

,title,genres,runtime,vote_average,vote_count,overview,id,year,score,cast,crew,keywords
0,Toy Story,"[animation, comedy, family]",81.0,7.7,5415.0,"Led by Woody, Andy's toys live happily in his ...",862,1995.0,7.680953,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,Jumanji,"[adventure, fantasy, family]",104.0,6.9,2413.0,When siblings Judy and Peter discover an encha...,8844,1995.0,6.873979,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,Grumpier Old Men,"[romance, comedy]",101.0,6.5,92.0,A family wedding reignites the ancient feud be...,15602,1995.0,6.189510,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,Father of the Bride Part II,[comedy],106.0,5.7,173.0,Just when George Banks has recovered from his ...,11862,1995.0,5.681661,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
4,Heat,"[action, crime, drama, thriller]",170.0,7.7,1886.0,"Obsessive master thief, Neil McCauley leads a ...",949,1995.0,7.646235,"[{'cast_id': 25, 'character': 'Lt. Vincent Han...","[{'credit_id': '52fe4292c3a36847f802916d', 'de...","[{'id': 642, 'name': 'robbery'}, {'id': 703, '..."


In [9]:
len(df.iloc[0]['crew'])

106

In [10]:
df.iloc[3]['crew']

[{'credit_id': '52fe44959251416c75039ed7',
  'department': 'Sound',
  'gender': 2,
  'id': 37,
  'job': 'Original Music Composer',
  'name': 'Alan Silvestri',
  'profile_path': '/chEsfnDEtRmv1bfOaNAoVEzhCc6.jpg'},
 {'credit_id': '52fe44959251416c75039ee9',
  'department': 'Camera',
  'gender': 2,
  'id': 5506,
  'job': 'Director of Photography',
  'name': 'Elliot Davis',
  'profile_path': None},
 {'credit_id': '52fe44959251416c75039ecb',
  'department': 'Writing',
  'gender': 1,
  'id': 17698,
  'job': 'Screenplay',
  'name': 'Nancy Meyers',
  'profile_path': '/nMPHU06dnvVxEjjcnPCPUQgQ2Mp.jpg'},
 {'credit_id': '52fe44959251416c75039edd',
  'department': 'Production',
  'gender': 1,
  'id': 17698,
  'job': 'Producer',
  'name': 'Nancy Meyers',
  'profile_path': '/nMPHU06dnvVxEjjcnPCPUQgQ2Mp.jpg'},
 {'credit_id': '52fe44959251416c75039ed1',
  'department': 'Writing',
  'gender': 2,
  'id': 26160,
  'job': 'Screenplay',
  'name': 'Albert Hackett',
  'profile_path': None},
 {'credit_id': '

In [11]:
def get_director(x):
    for crew_member in x:
        if crew_member['job'] == 'Director':
            return crew_member['name']
    return np.nan

df['director'] = df['crew'].apply(get_director)
df['director'].head()

0    John Lasseter
1     Joe Johnston
2    Howard Deutch
3    Charles Shyer
4     Michael Mann
Name: director, dtype: object

In [12]:
def get_producer(x):
    for crew_member in x:
        if crew_member['job'] == 'Producer':
            return crew_member['name']
    return np.nan

df['producer'] = df['crew'].apply(get_producer)
df['producer'].head()

0    Bonnie Arnold
1     Scott Kroopf
2              NaN
3     Nancy Meyers
4       Art Linson
Name: producer, dtype: object

In [13]:
max_number_of_actors = 3
def get_cast(cast):
    actors = []
    count = 0
    for cast_member in cast:
        if count<max_number_of_actors:
            actors.append(cast_member['name'].replace(" ",""))
        count += 1

    return actors

df['cast'] = df['cast'].apply(get_cast)

In [14]:
max_number_of_actors = 5
def get_keywords(keywords):
    keys = []
    count = 0
    for keyword in keywords:
        if count<max_number_of_actors:
            keys.append(keywords['name'])
        count += 1

    return keys

df['keywords'] = df['keywords'].apply(get_cast)

In [15]:
general_columns = ['cast', 'keywords', 'director', 'producer', 'genres']
df[general_columns]

,cast,keywords,director,producer,genres
0,"[TomHanks, TimAllen, DonRickles]","[jealousy, toy, boy, friendship, friends]",John Lasseter,Bonnie Arnold,"[animation, comedy, family]"
1,"[RobinWilliams, JonathanHyde, KirstenDunst]","[boardgame, disappearance, basedonchildren'sbo...",Joe Johnston,Scott Kroopf,"[adventure, fantasy, family]"
2,"[WalterMatthau, JackLemmon, Ann-Margret]","[fishing, bestfriend, duringcreditsstinger, ol...",Howard Deutch,NaN,"[romance, comedy]"
3,"[SteveMartin, DianeKeaton, MartinShort]","[baby, midlifecrisis, confidence, aging, daugh...",Charles Shyer,Nancy Meyers,[comedy]
4,"[AlPacino, RobertDeNiro, ValKilmer]","[robbery, detective, bank, obsession, chase]",Michael Mann,Art Linson,"[action, crime, drama, thriller]"
...,...,...,...,...,...
9292,"[ChristianClavier, AryAbittan, ElsaZylberstein]",[],Philippe de Chauveron,Patrice Ledoux,[comedy]
9293,"[JeanReno, ChristianClavier, FranckDubosc]","[nazis, castle, timetravel, robespierre]",Jean-Marie Poiré,NaN,[comedy]
9294,"[ShaneVanDyke, MarieWestbrook, BruceDavison]",[suspense],Shane Van Dyke,NaN,"[action, adventure, thriller]"
9295,[],"[love, teenager, lgbt, short]",Beth David,NaN,"[family, animation, romance, comedy]"


In [16]:
for i in general_columns:
    df[i] = df[i].fillna('')


In [17]:
df[general_columns]

,cast,keywords,director,producer,genres
0,"[TomHanks, TimAllen, DonRickles]","[jealousy, toy, boy, friendship, friends]",John Lasseter,Bonnie Arnold,"[animation, comedy, family]"
1,"[RobinWilliams, JonathanHyde, KirstenDunst]","[boardgame, disappearance, basedonchildren'sbo...",Joe Johnston,Scott Kroopf,"[adventure, fantasy, family]"
2,"[WalterMatthau, JackLemmon, Ann-Margret]","[fishing, bestfriend, duringcreditsstinger, ol...",Howard Deutch,,"[romance, comedy]"
3,"[SteveMartin, DianeKeaton, MartinShort]","[baby, midlifecrisis, confidence, aging, daugh...",Charles Shyer,Nancy Meyers,[comedy]
4,"[AlPacino, RobertDeNiro, ValKilmer]","[robbery, detective, bank, obsession, chase]",Michael Mann,Art Linson,"[action, crime, drama, thriller]"
...,...,...,...,...,...
9292,"[ChristianClavier, AryAbittan, ElsaZylberstein]",[],Philippe de Chauveron,Patrice Ledoux,[comedy]
9293,"[JeanReno, ChristianClavier, FranckDubosc]","[nazis, castle, timetravel, robespierre]",Jean-Marie Poiré,,[comedy]
9294,"[ShaneVanDyke, MarieWestbrook, BruceDavison]",[suspense],Shane Van Dyke,,"[action, adventure, thriller]"
9295,[],"[love, teenager, lgbt, short]",Beth David,,"[family, animation, romance, comedy]"


In [18]:
for col in general_columns:
    df[col] = df[col].replace('\d+', '')

In [19]:
df[general_columns]

,cast,keywords,director,producer,genres
0,"[TomHanks, TimAllen, DonRickles]","[jealousy, toy, boy, friendship, friends]",John Lasseter,Bonnie Arnold,"[animation, comedy, family]"
1,"[RobinWilliams, JonathanHyde, KirstenDunst]","[boardgame, disappearance, basedonchildren'sbo...",Joe Johnston,Scott Kroopf,"[adventure, fantasy, family]"
2,"[WalterMatthau, JackLemmon, Ann-Margret]","[fishing, bestfriend, duringcreditsstinger, ol...",Howard Deutch,,"[romance, comedy]"
3,"[SteveMartin, DianeKeaton, MartinShort]","[baby, midlifecrisis, confidence, aging, daugh...",Charles Shyer,Nancy Meyers,[comedy]
4,"[AlPacino, RobertDeNiro, ValKilmer]","[robbery, detective, bank, obsession, chase]",Michael Mann,Art Linson,"[action, crime, drama, thriller]"
...,...,...,...,...,...
9292,"[ChristianClavier, AryAbittan, ElsaZylberstein]",[],Philippe de Chauveron,Patrice Ledoux,[comedy]
9293,"[JeanReno, ChristianClavier, FranckDubosc]","[nazis, castle, timetravel, robespierre]",Jean-Marie Poiré,,[comedy]
9294,"[ShaneVanDyke, MarieWestbrook, BruceDavison]",[suspense],Shane Van Dyke,,"[action, adventure, thriller]"
9295,[],"[love, teenager, lgbt, short]",Beth David,,"[family, animation, romance, comedy]"


In [29]:
def get_iterable(row):
    if isinstance(row, list):
        return [str.lower(i.replace(" ", "")) for i in row]
    else:
        if isinstance(row, str):
            return str.lower(row.replace(" ", ""))
        else:
            return ""

for feature in general_columns:
    df[feature] = df[feature].apply(get_iterable)

In [30]:
df[general_columns]

,cast,keywords,director,producer,genres
0,"[tomhanks, timallen, donrickles]","[jealousy, toy, boy, friendship, friends]",johnlasseter,bonniearnold,"[animation, comedy, family]"
1,"[robinwilliams, jonathanhyde, kirstendunst]","[boardgame, disappearance, basedonchildren'sbo...",joejohnston,scottkroopf,"[adventure, fantasy, family]"
2,"[waltermatthau, jacklemmon, ann-margret]","[fishing, bestfriend, duringcreditsstinger, ol...",howarddeutch,,"[romance, comedy]"
3,"[stevemartin, dianekeaton, martinshort]","[baby, midlifecrisis, confidence, aging, daugh...",charlesshyer,nancymeyers,[comedy]
4,"[alpacino, robertdeniro, valkilmer]","[robbery, detective, bank, obsession, chase]",michaelmann,artlinson,"[action, crime, drama, thriller]"
...,...,...,...,...,...
9292,"[christianclavier, aryabittan, elsazylberstein]",[],philippedechauveron,patriceledoux,[comedy]
9293,"[jeanreno, christianclavier, franckdubosc]","[nazis, castle, timetravel, robespierre]",jean-mariepoiré,,[comedy]
9294,"[shanevandyke, mariewestbrook, brucedavison]",[suspense],shanevandyke,,"[action, adventure, thriller]"
9295,[],"[love, teenager, lgbt, short]",bethdavid,,"[family, animation, romance, comedy]"


In [32]:
def summirize_features(features):
    return ' '.join(features['keywords']) + ' ' + ' '.join(features['cast']) + ' ' + features['director'] + ' ' + ' '.join(features['producer']) + ' ' + ' '.join(features['genres'])

df['feature'] = df.apply(summirize_features, axis=1)
df['feature']

0       jealousy toy boy friendship friends tomhanks t...
1       boardgame disappearance basedonchildren'sbook ...
2       fishing bestfriend duringcreditsstinger oldmen...
3       baby midlifecrisis confidence aging daughter s...
4       robbery detective bank obsession chase alpacin...
                              ...                        
9292     christianclavier aryabittan elsazylberstein p...
9293    nazis castle timetravel robespierre jeanreno c...
9294    suspense shanevandyke mariewestbrook brucedavi...
9295    love teenager lgbt short  bethdavid  family an...
9296    militaryschool hilaryduff christycarlsonromano...
Name: feature, Length: 9297, dtype: object

In [23]:
# def summirize_features(df):
#     important_features = []
#     for index, row in df.iterrows():
#         new_col = row['cast'] + ' ' + row['keywords'] + ' '  + row['director'] + ' '  + row['producer'] + ' '  + row['genres']
#         important_features.append(new_col.lower())

#     return important_features
    

In [33]:
vect = TfidfVectorizer(stop_words = 'english')
vect_matrix = vect.fit_transform(df['feature'])
feature_names = vect.get_feature_names()

print('Длина словаря: ' + str(len(feature_names)))

cs = cosine_similarity(vect_matrix, vect_matrix)

df = df.reset_index()
indices = pd.Series(df.index, index=df['title'])

c:\Users\Sveta\.conda\envs\recommmmenadtion_system\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Длина словаря: 21228


In [47]:
def get_recommendations(title, cosine_sim):
    idx = indices[title]
    similarity_scores = list(enumerate(cosine_sim[idx]))
    similarity_scores= sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores= similarity_scores[1:11]
    # (a, b) where a is id of movie, b is similarity_scores
    movies_indices = [ind[0] for ind in similarity_scores]
    movies = df["title"].iloc[movies_indices]
    return movies

In [49]:
# print("Рекомендации для Avengers: ")
# print(get_recommendations("The Avengers", cs))